<a href="https://colab.research.google.com/github/PedroLuizPG/Processamento-de-Imagens_E01_Grupo3/blob/feature%2Fcomparar_imagens/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from skimage.transform import rotate
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from datetime import datetime
from IPython.display import display, clear_output, Markdown
from PIL import Image
import pandas as pd
from google.colab import files
import zipfile
import ipywidgets as widgets

PASTA_SAIDA = "imagens_processadas"
os.makedirs(PASTA_SAIDA, exist_ok=True)

historico = []
imagens = []

doc_md = """
# Módulo: Redimensionamento / Rotação / Auto-align / Miniaturas

**O que este notebook fornece (resumo):**
- Upload de imagens pelo Colab.
- Pré-visualização em tempo real das transformações.
- Redimensionamento manual/automático (preserva proporção).
- Rotação manual (ângulo) e rotação automática (estimativa via HoughLinesP).
- Recorte automático usando detecção de contornos (ROI).
- `auto_align`: recorte + correção de inclinação (pipeline automático).
- Geração automática de miniaturas (vários tamanhos) e salvamento.
- Histórico e exportação ZIP dos resultados.

Use a interface abaixo para carregar suas imagens, ajustar parâmetros e confirmar o processamento.
"""
display(Markdown(doc_md))

def upload_imagens_colab():
    """Upload de imagens no Colab e retorna lista de (caminho, imagem)."""
    uploaded = files.upload()
    imgs = []
    for nome in uploaded.keys():
        caminho = nome
        img = cv2.imread(caminho)
        if img is not None:
            imgs.append((caminho, img))
        else:
            print(f"Erro ao carregar {caminho}")
    return imgs

def validar_imagem(img):
    return img is not None and img.shape[0] > 10 and img.shape[1]

def redimensionar_imagem(img, largura_alvo=None, altura_alvo=None):
    """Redimensiona mantendo proporção aproximada."""
    h, w = img.shape[:2]
    if largura_alvo and altura_alvo:
        proporcao = min(largura_alvo / w, altura_alvo / h)
        novo_w = int(w * proporcao)
        novo_h = int(h * proporcao)
    elif largura_alvo:
        proporcao = largura_alvo / w
        novo_w = largura_alvo
        novo_h = int(h * proporcao)
    elif altura_alvo:
        proporcao = altura_alvo / h
        novo_w = int(w * proporcao)
        novo_h = altura_alvo
    else:
        return img
    return cv2.resize(img, (novo_w, novo_h), interpolation=cv2.INTER_CUBIC)

def redimensionar_automatico(img, largura_max=800, altura_max=800):
    """Redimensiona automaticamente mantendo proporção."""
    h, w = img.shape[:2]
    proporcao = min(largura_max / w, altura_max / h, 1.0)
    novo_w, novo_h = int(w * proporcao), int(h * proporcao)
    return cv2.resize(img, (novo_w, novo_h), interpolation=cv2.INTER_CUBIC)

def rotacionar_imagem(img, angulo=0):
    """Rotação manual (skimage.rotate) preservando range."""
    if img is None:
        return img
    return rotate(img, angle=angulo, resize=True, preserve_range=True).astype(np.uint8)

def rotacao_automatica(img, debug=False):
    """
    Detecta o ângulo dominante com HoughLinesP (linhas detectadas nas bordas)
    e corrige a inclinação.
    Retorna a imagem rotacionada. Se não houver linhas suficientes, retorna original.
    """
    if img is None:
        return img

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img.copy()
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150, apertureSize=3)

    linhasP = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)
    if linhasP is None:
        if debug: print("rotacao_automatica: sem linhas detectadas")
        return img

    angulos = []
    for l in linhasP:
        x1, y1, x2, y2 = l[0]
        dx = x2 - x1
        dy = y2 - y1
        if dx == 0:
            continue
        ang = np.degrees(np.arctan2(dy, dx))
        if abs(ang) < 45:
            angulos.append(ang)

    if not angulos:
        if debug: print("rotacao_automatica: sem ângulos úteis")
        return img

    ang_mediana = np.median(angulos)
    if debug:
        print(f"rotacao_automatica: angulos detectados (mediana) = {ang_mediana:.2f} deg")

    img_corrigida = rotacionar_imagem(img, -ang_mediana)
    return img_corrigida

def recorte_automatico(img, padding=5):
    """
    Detecta a maior ROI por contornos após threshold adaptativo e retorna o recorte.
    Se não encontrar contornos relevantes, retorna a imagem original.
    """
    if img is None:
        return img
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img.copy()
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    clean = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)

    contornos, _ = cv2.findContours(clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contornos:
        return img

    maior = max(contornos, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(maior)

    x0 = max(x - padding, 0)
    y0 = max(y - padding, 0)
    x1 = min(x + w + padding, img.shape[1])
    y1 = min(y + h + padding, img.shape[0])

    return img[y0:y1, x0:x1]

def auto_align(img, debug=False):
    """
    Pipeline automático: recorte por ROI -> detecção de inclinação -> correção.
    Retorna imagem recortada e alinhada.
    """
    if img is None:
        return img
    rec = recorte_automatico(img)
    rot = rotacao_automatica(rec, debug=debug)
    return rot

def gerar_miniaturas(img, tamanhos=[(64,64), (128,128), (256,256)], prefix="thumb"):
"""
Gera miniaturas (thumbnails) com interpolação AREA (boa para redução),
salva em PASTA_SAIDA e retorna lista de caminhos salvos.
"""
  thumbs_paths = []
  for w, h in tamanhos:
      thumb = cv2.resize(img, (w,h), interpolation=cv2.INTER_AREA)
      nome = f"{prefix}_{w}x{h}.jpg"
      caminho = os.path.join(PASTA_SAIDA, nome)
      cv2.imwrite(caminho, thumb, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
      thumbs_paths.append(caminho)
  return thumbs_paths

def comparar_imagens(img1, img2, titulo1="Original", titulo2="Processada"):
    plt.figure(figsize=(10,5))
    for i, (im, title) in enumerate(zip([img1, img2], [titulo1, titulo2]), 1):
        plt.subplot(1,2,i)
        if im is None:
            plt.text(0.5, 0.5, "Imagem vazia", ha='center')
            continue
        if len(im.shape) == 3:
            plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        else:
            plt.imshow(im, cmap='gray')
        plt.title(title)
        plt.axis('off')
    plt.show()


# Módulo: Redimensionamento / Rotação / Auto-align / Miniaturas

**O que este notebook fornece (resumo):**
- Upload de imagens pelo Colab.
- Pré-visualização em tempo real das transformações.
- Redimensionamento manual/automático (preserva proporção).
- Rotação manual (ângulo) e rotação automática (estimativa via HoughLinesP).
- Recorte automático usando detecção de contornos (ROI).
- `auto_align`: recorte + correção de inclinação (pipeline automático).
- Geração automática de miniaturas (vários tamanhos) e salvamento.
- Histórico e exportação ZIP dos resultados.

Use a interface abaixo para carregar suas imagens, ajustar parâmetros e confirmar o processamento.
